# Bank Churners
**ML Project 1 - Spring 2022**

Collaborators: Ben DeSollar and Matt McDonnell

Descriptions: The first part should be a description which models/classifiers/regressors achieved the best 
    results with your datasets datasets, and what you did to improve those results along the way.

    - Through research we found that a called RandomForestClassifier worked the best after testing a couple different model including
    Niave Bayes and DecisionTreeClassifier. We improved our results by modifying our feature selection to include as many features as we 
    could to help predict the best values. Things like gender and Total relationship count did not seem as impptant as education level or
    credit limit. By modifying our featuiues during testing we were able to find the best model predictions.



### Step 1
Import the necessary packages and frameworks.

In [10]:
import numpy as np
from mlwpy import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics,
                     pipeline,
                     preprocessing as skpre) 
import csv
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from seaborn import load_dataset
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder

### Step 2
Read in .csv files and save as DataFrames.

In [11]:
data_train_df = pd.read_csv("cars_class_train.csv")#, nrows=86108) 
data_test_df = pd.read_csv("cars_class_test.csv")#, nrows=86108)
data_train_ft = data_train_df.drop('type', axis=1)
data_train_tgt = data_train_df["type"]

### Step 3
Work with only the necessary features. We tried to choose features that were only about the users finances and not cateogies like gender or relationship count.
Yet things like marital status and education level seemed to have an impact on the prediction scores. 

In [12]:
# To see what kind of data we're working with
print(data_test_df.dtypes)
print(data_train_ft.dtypes)
# print(data_train_ft['manufacturer'].unique())
# print(data_train_ft.head(5))

id                int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
paint_color      object
image_url        object
description      object
state            object
lat             float64
long            float64
posting_date     object
dtype: object
Unnamed: 0        int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
pa

### Step 4
Work with a practice estimate (median age). Convert features to numerical values as necessary. 

For now, let's use a simple approach to estimate the age (and consider revisiting this estimate later) when it is missing by using the median from the other samples.

We also replaced any missing data in the dataset.

In [13]:

# 5 6 8 12 13

features = ['year',\
            'odometer',
            'lat',
            'long',
            'drive',
            'size',
            #'posting_date',
            'fuel',
            'condition',
            'model',
            'paint_color',
            'cylinders',
            'manufacturer',
            'VIN',
            'state',
            'price'
            ]

data_train_ft = data_train_ft[features]
data_train_ft = data_train_ft.fillna(0)

data_test_ft = data_test_df
data_test_df = data_test_df[features]
data_test_df = data_test_df.fillna(0)

'''
TE_encoder = TargetEncoder()
data_train_ft = TE_encoder.fit_transform(data_train_ft, data_train_tgt)
data_test_df = TE_encoder.transform(data_test_df)
'''
'''
LOOE_encoder = LeaveOneOutEncoder()
data_train_ft = LOOE_encoder.fit_transform(data_train_ft, data_train_tgt)
data_test_df = LOOE_encoder.transform(data_test_df)
'''

LE_encoder = OrdinalEncoder(features)
data_train_ft = LE_encoder.fit_transform(data_train_ft)
data_test_df = LE_encoder.transform(data_test_df)
data_train_ft.head()

'''
ftrsel = ftr_sel.SelectKBest(ftr_sel.f_classif, k=5)
ftrsel.fit_transform(data_train_ft, data_train_tgt)

keepers_idx = ftrsel.get_support()
print(keepers_idx)
# use target encoding to encode two categorical features

ftrsel = ftr_sel.SelectFromModel(ensemble.RandomForestClassifier(), 
                                 threshold='mean') # default

ftrsel = ftr_sel.SelectKBest(ftr_sel.f_classif, 
                                  k=5)
'''
ftrsel = ftr_sel.RFE(ensemble.RandomForestClassifier(),
                     n_features_to_select=7)
ftrsel.fit_transform(data_train_ft, data_train_tgt)

print(np.array(features)[ftrsel.get_support()])
new_features = np.array(features)[ftrsel.get_support()]
data_train_ft = data_train_ft[new_features]
data_test_df = data_test_df[new_features]

['odometer' 'lat' 'long' 'drive' 'model' 'manufacturer' 'price']


Here is an updated table after sorting through these features.

In [14]:
print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())

data_train_ft = data_train_ft.astype(float)
data_test_df = data_test_df.astype(float)

print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())

# Analysis of the features looks like Education level, Credit Limit, and Income Level may be useful. 
# Features will need to be transformed to convert all into numbers to be analyzed.

data_train_ft:


,odometer,lat,long,drive,model,manufacturer,price
0,"155,815.0000",41.6002,-93.5701,1,1,1,8995
1,"72,953.0000",40.3074,-79.5424,2,2,2,4500
2,0.0000,35.2468,-120.6676,2,3,3,16495
3,"64,855.0000",35.7489,-78.6434,3,4,4,14900
4,"10,873.0000",43.0172,-82.4510,4,5,5,35990
5,"97,465.0000",39.8264,-76.9991,1,6,3,12900
6,"90,360.0000",41.3900,-83.6511,3,7,2,6999
7,"84,317.0000",40.4688,-74.2817,2,8,6,34990
8,"36,400.0000",40.5829,-73.9679,3,9,7,8975
9,"117,142.0000",41.7262,-71.3347,3,10,8,4000


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200916 entries, 0 to 200915
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   odometer      200916 non-null  float64
 1   lat           200916 non-null  float64
 2   long          200916 non-null  float64
 3   drive         200916 non-null  int64  
 4   model         200916 non-null  int64  
 5   manufacturer  200916 non-null  int64  
 6   price         200916 non-null  int64  
dtypes: float64(3), int64(4)
memory usage: 10.7 MB


None

data_train_ft:


,odometer,lat,long,drive,model,manufacturer,price
0,"155,815.0000",41.6002,-93.5701,1.0000,1.0000,1.0000,"8,995.0000"
1,"72,953.0000",40.3074,-79.5424,2.0000,2.0000,2.0000,"4,500.0000"
2,0.0000,35.2468,-120.6676,2.0000,3.0000,3.0000,"16,495.0000"
3,"64,855.0000",35.7489,-78.6434,3.0000,4.0000,4.0000,"14,900.0000"
4,"10,873.0000",43.0172,-82.4510,4.0000,5.0000,5.0000,"35,990.0000"
5,"97,465.0000",39.8264,-76.9991,1.0000,6.0000,3.0000,"12,900.0000"
6,"90,360.0000",41.3900,-83.6511,3.0000,7.0000,2.0000,"6,999.0000"
7,"84,317.0000",40.4688,-74.2817,2.0000,8.0000,6.0000,"34,990.0000"
8,"36,400.0000",40.5829,-73.9679,3.0000,9.0000,7.0000,"8,975.0000"
9,"117,142.0000",41.7262,-71.3347,3.0000,10.0000,8.0000,"4,000.0000"


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200916 entries, 0 to 200915
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   odometer      200916 non-null  float64
 1   lat           200916 non-null  float64
 2   long          200916 non-null  float64
 3   drive         200916 non-null  float64
 4   model         200916 non-null  float64
 5   manufacturer  200916 non-null  float64
 6   price         200916 non-null  float64
dtypes: float64(7)
memory usage: 10.7 MB


None

### Step 5
Convert all feature values to floating-point numbers for better accuracy when performing tests later.

### Step 6
Create a library of different models to train and track the accuracy of later. Like the feature selection, having a wide variety of models to test out helps us ensure that our predictions are as accurate as possible.

In [15]:
pipelines_to_try = {'nb': naive_bayes.GaussianNB(),\
                    'GNB' : naive_bayes.GaussianNB(),
                    'RandomForestClassifier' : RandomForestClassifier()
                    }

# add k-NN models with various values of k to models_to_try
for k in range(1,6,2):
    pipelines_to_try[f'{k}-NN'] = neighbors.KNeighborsClassifier(n_neighbors=k)

In [16]:
scaler = skpre.StandardScaler()
for k in range(1,6,2):
    pipelines_to_try[f'DTC-{k}'] =  tree.DecisionTreeClassifier(max_depth=k)

baseline = dummy.DummyClassifier(strategy="uniform")
pipelines_to_try1 = {}
# scale the data
for model_name in pipelines_to_try:
    pipelines_to_try1[f'std_{model_name}_pipe'] = pipeline.make_pipeline(scaler, 
                                                      pipelines_to_try[model_name])


sv_classifiers = {"SVC(Linear)"   : svm.SVC(kernel='linear'),
                  "NuSVC(Linear)" : svm.NuSVC(kernel='linear', nu=.9)} 

### Step 8
Use cross-validation to test the accuracy of the models.

In [17]:
accuracy_scores = {}
loo = skms.LeaveOneOut()
for name, model in pipelines_to_try1.items():
    loo = skms.LeaveOneOut()
    print(f'Currently modeling using : {name}')
    scores = skms.cross_val_score(model,
                                  data_train_ft,
                                  data_train_tgt,
                                  #cv=loo,
                                  #n_jobs=-1,
                                  cv=10,
                                  scoring='accuracy')
    mean_accuracy = scores.mean()
    accuracy_scores[name] = mean_accuracy
    print(f'{name}: {mean_accuracy:.3f}')

Currently modeling using : std_nb_pipe
std_nb_pipe: 0.239
Currently modeling using : std_GNB_pipe
std_GNB_pipe: 0.239
Currently modeling using : std_RandomForestClassifier_pipe


### Step 9
Find the pipeline with the highest accuracy and save it for testing.

In [ ]:
best_pipeline_name = max(accuracy_scores,key=accuracy_scores.get)
print(f'\nBest pipeline: {best_pipeline_name} (accuracy = {accuracy_scores[best_pipeline_name]:.3f})')
final_pipeline = pipelines_to_try1[best_pipeline_name]

# Best model seems to be RandomForestClassifier (accuracy = 0.954)


Best pipeline: std_RandomForestClassifier_pipe (accuracy = 0.563)


### Step 10
Fit and test the highest-scoring model against the testing data. Save the results in submission.csv for uploading to Kaggle.

In [ ]:
scaler.fit(data_train_ft)

train_scaled_ftrs = scaler.transform(data_train_ft)

# transform validation features
test_scaled_ftrs = scaler.transform(data_test_df)

fit = final_pipeline.fit(train_scaled_ftrs, data_train_tgt)
predictions = fit.predict(test_scaled_ftrs)
#fit = final_pipeline.fit(data_train_ft, data_train_tgt)
#predictions = fit.predict(data_test_df)
make_submission_df = pd.read_csv("cars_class_test.csv")
# drop all columns except 'id'
make_submission_df = make_submission_df[['id']]
# make sure the column of ID's that we just read in is the index column
make_submission_df = make_submission_df.set_index('id')
# write to csv file
make_submission_df['type'] = predictions
make_submission_df.to_csv('submission.csv',sep=',')
# display message
print("Saved predictions to csv file.")

KeyboardInterrupt: 